In [120]:
# Import packages
import pandas as pd
import numpy as np
from google.colab import files

In [121]:
# Getting the application record datasets
url = "https://raw.githubusercontent.com/manish-cr/CS3244-credit-approval/master/data/application_record.csv"
df = pd.read_csv(url)

In [122]:
# Get to know the datasets
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [123]:
df.describe()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS
count,4.385570e+05,438557.000000,4.385570e+05,438557.000000,438557.000000,438557.0,438557.000000,438557.000000,438557.000000,438557.000000
mean,6.022176e+06,0.427390,1.875243e+05,-15997.904649,60563.675328,1.0,0.206133,0.287771,0.108207,2.194465
std,5.716370e+05,0.724882,1.100869e+05,4185.030007,138767.799647,0.0,0.404527,0.452724,0.310642,0.897207
min,5.008804e+06,0.000000,2.610000e+04,-25201.000000,-17531.000000,1.0,0.000000,0.000000,0.000000,1.000000
25%,5.609375e+06,0.000000,1.215000e+05,-19483.000000,-3103.000000,1.0,0.000000,0.000000,0.000000,2.000000
50%,6.047745e+06,0.000000,1.607805e+05,-15630.000000,-1467.000000,1.0,0.000000,0.000000,0.000000,2.000000
75%,6.456971e+06,1.000000,2.250000e+05,-12514.000000,-371.000000,1.0,0.000000,1.000000,0.000000,3.000000
max,7.999952e+06,19.000000,6.750000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000


We can see that the column FLAG_MOBIL is all 1. Thus we will drop this column to reduce the dimension of the dataset.

In [124]:
# Dropping the FLAG_MOBIL column
df = df.drop(columns='FLAG_MOBIL')

In [125]:
# Checking for multiple applications by same customer
df['ID'].duplicated().sum()

47

There are 47 duplicates. We will remove the duplicates and keep the latest application.

In [126]:
# Sorting by birthdates
# so the latest will come first
df = df.sort_values(by='DAYS_BIRTH').drop_duplicates(subset='ID')

In [127]:
# Checking for empty values
df.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134197
CNT_FAM_MEMBERS             0
dtype: int64

In [128]:
# Types of income
df['NAME_INCOME_TYPE'].unique()

array(['Pensioner', 'Working', 'Commercial associate', 'State servant',
       'Student'], dtype=object)

In [129]:
# Types of occupation
df['OCCUPATION_TYPE'].unique()

array([nan, 'Laborers', 'High skill tech staff', 'Realty agents',
       'Managers', 'Cleaning staff', 'Core staff', 'Sales staff',
       'Accountants', 'Security staff', 'Medicine staff', 'Drivers',
       'Private service staff', 'Cooking staff', 'Low-skill Laborers',
       'Secretaries', 'Waiters/barmen staff', 'IT staff', 'HR staff'],
      dtype=object)

From this point, we will split the dataset into two, those who are employed and unemployed.


---



# DAYS_EMPLOYED >= 0

In [130]:
# Splitting the data
unemployed_df = df[df['DAYS_EMPLOYED']>=0]
# Income type of those that are not emplyed
unemployed_df['NAME_INCOME_TYPE'].unique()

array(['Pensioner'], dtype=object)

From the table above, we can see that there are applications with DAYS_EMPLOYED being 365243 days. This suggests that they have been unemplyed for 1000 years which are not accurate and will affect with the scaling of the dataset. We will thus need to edit the days accordingly.

We have also realised that those who are not employed are all pensioners. Thus we will set their occuption type to pensioners if they are intially missing.

We will set the lower 25percentile of the age group as retirement age. The days of unemployment will be their age minus the retirement age or their current age.



In [131]:
# Age distribution
df['DAYS_BIRTH'].describe()

count    438510.000000
mean     -15998.192778
std        4185.074780
min      -25201.000000
25%      -19484.000000
50%      -15630.000000
75%      -12514.000000
max       -7489.000000
Name: DAYS_BIRTH, dtype: float64

In [132]:
# Filling up the missing values
unemployed_df = unemployed_df.fillna(value={'OCCUPATION_TYPE':'Pensioner'})

In [133]:
# Adjusting the DAYS_EMPLOYED
retirement_age = -19484       ## From the lower 25percentile of the age range

unemployed_df['DAYS_EMPLOYED'] = unemployed_df.apply(lambda x: (retirement_age-x['DAYS_BIRTH']) if (x['DAYS_BIRTH']<retirement_age) else (-(x['DAYS_BIRTH'])), axis=1)

# DAYS_EMPLOYED < 0

In [134]:
# Splitting up the data
employed_df = df[df['DAYS_EMPLOYED']<0]

In [135]:
# Checking for missing values
employed_df.isnull().sum()

ID                         0
CODE_GENDER                0
FLAG_OWN_CAR               0
FLAG_OWN_REALTY            0
CNT_CHILDREN               0
AMT_INCOME_TOTAL           0
NAME_INCOME_TYPE           0
NAME_EDUCATION_TYPE        0
NAME_FAMILY_STATUS         0
NAME_HOUSING_TYPE          0
DAYS_BIRTH                 0
DAYS_EMPLOYED              0
FLAG_WORK_PHONE            0
FLAG_PHONE                 0
FLAG_EMAIL                 0
OCCUPATION_TYPE        58868
CNT_FAM_MEMBERS            0
dtype: int64

In [136]:
# Checking the income type
employed_df['NAME_INCOME_TYPE'].unique()

array(['Working', 'Commercial associate', 'State servant', 'Pensioner',
       'Student'], dtype=object)

Although they are working, there are still some with income type Pensioner. In this case we will just label all missing values with Unknown.

In [137]:
# Filling up the missing values
employed_df = employed_df.fillna(value={'OCCUPATION_TYPE':'Unknown'})

In [138]:
# Combining the two sub dataset
cleaned_df = pd.concat([unemployed_df, employed_df]).sort_values(by='ID')

In [139]:
cleaned_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,0,0,Unknown,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,0,0,Unknown,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,0,1,1,Sales staff,1.0


In [140]:
cleaned_df.describe()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS
count,4.385100e+05,438510.000000,4.385100e+05,438510.000000,438510.000000,438510.000000,438510.000000,438510.000000,438510.000000
mean,6.022035e+06,0.427372,1.875239e+05,-15998.192778,-1508.382812,0.206121,0.287761,0.108200,2.194449
std,5.714962e+05,0.724870,1.100881e+05,4185.074780,3739.995157,0.404519,0.452720,0.310634,0.897187
min,5.008804e+06,0.000000,2.610000e+04,-25201.000000,-17531.000000,0.000000,0.000000,0.000000,1.000000
25%,5.609362e+06,0.000000,1.215000e+05,-19484.000000,-3103.000000,0.000000,0.000000,0.000000,2.000000
50%,6.047720e+06,0.000000,1.611000e+05,-15630.000000,-1467.000000,0.000000,0.000000,0.000000,2.000000
75%,6.454161e+06,1.000000,2.250000e+05,-12514.000000,-371.000000,0.000000,1.000000,0.000000,3.000000
max,7.999952e+06,19.000000,6.750000e+06,-7489.000000,19482.000000,1.000000,1.000000,1.000000,20.000000


In [141]:
cleaned_df.isnull().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
dtype: int64

In [117]:
# Downloading the cleaned dataset
#cleaned_df.to_csv('application_cleaned.csv', encoding = 'utf-8-sig')
#files.download('application_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>